<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction to assignment

The goal of this assignment is to show neighborhoods in Toronto through exploring, segmenting and clustering. 
First, the data will be scraped from Wikipedia page. After that, it will be preprocesed and cleaned, prepered for futher work. In this work, it will be converted addresses into their equivalent latitude and longitude values. Also, it will be used the Foursquare API to explore neighborhoods in Toronto.
There will be useed the k-means clustering algorithm, and the Folium library to visualize the neighborhoods in Toronto.

### Libraries

Downloading all dependencies that will be required.

In [1]:
import pandas as pd #library for data analsysis
import numpy as np #library to handle data in a vectorized manner

!pip install geopy #instal geopy
from geopy.geocoders import Nominatim #library to convert an address into latitude and longitude values

import requests #library to handle requests
import json #library to handle JSON files
from pandas.io.json import json_normalize #library to tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #library to import k-means from clustering stage

!pip install folium #instal folium
import folium #map rendering library

import urllib.request #library to use to open URLs

from bs4 import BeautifulSoup #library to import the BeautifulSoup library so we can parse HTML and XML documents

from sklearn.cluster import KMeans #library to import k-means from clustering stage

#libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Libraries imported.')

     |████████████████████████████████| 94 kB 4.8 MB/s  eta 0:00:01
Libraries imported.


### Downloading dataset and reading data

I tried to use the simplest and fastest way to get to the data and process it. That is why I use pandas to read html, to read, clean and preprocesed data.

In [2]:
#Specifying the url of the Wikipedia page is going to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Opening the url and puting the html into data_url variable. Class 'urllib.request.urlopener()' is base class for opening and reading URLs.
data_url = urllib.request.urlopen(url)

In [3]:
#Reading file from data_url and put them into readable table
data_toronto = pd.read_html(data_url)
neighborhood = data_toronto[0]
neighborhood

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
print('Shape of scraped data:', neighborhood.shape)

Shape of scraped data: (180, 3)


### Data preprocessing and cleaning

I think it is good way to clean data using pandas, to remove rows where is written 'not assigned' in th Borough column.
Setting index on some column allow to check data on that column and do grouping and joining for the neighborhood.
Reseting index will put default index which is integer and it will start from 0 and go to the lenght of data (rows).

In [5]:
#Droping "Not assigned" rows, they're not needed for analysis
neighborhood = neighborhood[neighborhood.Borough !='Not assigned']
neighborhood

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
#Grouping neighborhoods with the same postal code
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
merge_neighborhood = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)
merge_neighborhood

,,Neighbourhood
Postal Code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [7]:
# Resetting the index
new_neighborhood = merge_neighborhood.reset_index()
new_neighborhood

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In case that can be found 'Not assigned' in the column  'Neighborhood', the name of the Borough will replace 'Not assigned' in the column 'Neighborhood'

In [8]:
#Giving the borough's value to the neighborhood if is not assigned
new_name = new_neighborhood['Neighbourhood']
new_neighborhood[['Borough', 'Neighbourhood']]
borough_name = new_neighborhood['Borough']

for x in new_name:
    if x == 'Not assigned':
        y = borough_name
        z = new_neighborhood['Neighbourhood'].replace([x], [y])
        new_neighborhood['Neighbourhood'] = z
    else:
        None

In [9]:
new_neighborhood

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
#Showing the dataframe with all assigned neighborhoods' names
neighborhood_all = pd.DataFrame(new_neighborhood)
neighborhood_all

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
neighborhood_all.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#Printing shape of all neighborhood
print("Shape of all listed neighborhood:" , neighborhood_all.shape)

Shape of all listed neighborhood: (103, 3)


### Geospatial data and coordinates

To be able to see neighborhoods and interested places in the neighborhoods, I will import and read geospatila data, with latitude and longitude.

In [13]:
#Adding Geographical Coordinates from csv
data_geo = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
data_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437




Existing prepared table with neighborhoods and boroughs will be concatenated with new table with latitude and longitude, and joined over Postal Code.

In [14]:
#Cancatenation of neighborhood_all i data_geo
#Merging the two data sets over two columns on 'Postal Code'
merge_columns=pd.merge(neighborhood_all, data_geo, left_on='Postal Code', right_on='Postal Code')
merge_columns

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Here, I am checking how many are there Boroughs contain 'Toronto' in the name

In [15]:
#Show rows that contain 'Toronto' in column 'Borough'
neigh_toronto = merge_columns[merge_columns['Borough'].str.contains("Toronto")]
#Making dataframe
neighborhood_toronto = pd.DataFrame(neigh_toronto)
neighborhood_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [16]:
#Showing shape of neighborhood only with the name 'Toronto'
print("Shape of neighborhood that contains 'Toronto' in borough:" , neighborhood_toronto.shape)

Shape of neighborhood that contains 'Toronto' in borough: (39, 5)


I use geopy library to get lat and long values of Toronto

In [17]:
#Using geopy library to get latitude and longitude values of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating a map of Toronto

Now, that I have coordinates, I can create map of Toronto using folium.
Also, markers will be added to the maps, that will show the places according to the existinng latitude and longitude in the table.

In [18]:
#Creating a map of Toronto using latitude and longitude values

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#Adding markers to map

for lat, lng, postalcode, borough, neighbourhood in zip(neighborhood_toronto['Latitude'],neighborhood_toronto['Longitude'],neighborhood_toronto['Postal Code'],neighborhood_toronto['Borough'],neighborhood_toronto['Neighbourhood']):
    label = '{}, {}, {}'.format(postalcode, neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

### Segment and cluster only neighborhood in Downtown Toronto

In this part, I create map for one borough in Toronto, Downtown Toronto (it sounds like important part of Toronto).
First, I'll create dataframe with all data for Downtown Toronto, and found latitude and longitude for that part of Toronto. Based on that, map with marks on it will be created.

In [19]:
#Specifying exact borough which neighborhood will be checked
downtown_toronto_data = neighborhood_toronto[neighborhood_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [20]:
#Using geopy library to get latitude and longitude values of Toronto
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [21]:
#Creating a map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location = [latitude, longitude], zoom_start = 13)

#Adding markers to map
for lat, lng, postalcode, borough, neighbourhood in zip(downtown_toronto_data['Latitude'],downtown_toronto_data['Longitude'],downtown_toronto_data['Postal Code'],downtown_toronto_data['Borough'],downtown_toronto_data['Neighbourhood']):
    label = '{}, {}, {}'.format(postalcode, neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup=label,
    color='red',
    fill=True,
    fill_color='pink', #'#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_downtown_toronto)

map_downtown_toronto

### Toronto Neighbourhood and Foursquare

In [22]:
#Difining Foursquare credentials

In [48]:
CLIENT_ID = 'XUFOWH3UQYVE3QCEKH1BNRQCVFCDVMBB2IY0LHOUHQUGB3MX' # 4sq ID
CLIENT_SECRET = '114MEHQ12CFESHHTELECYJY5O3GOYHXDC2ARL1WQ0TWNNABD' # 4sq Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Exploring the first neighborhood in Torontno dataframe

In this part, I'll serach for neighborhoods and that will be inclulded in part of getting latitude and longitude values for that neighborhood, which will be print.
After that, I use function to make get request to the given location at foursquare, because that'll provide number of venus to the defining places. Using defining function with obtained latitude and longitude and neighborhood to get numbers of venues, and to group neighborhood by venues number, which could be used later.

In [23]:
#Getting the neighborhood's name
downtown_toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [24]:
#Getting neighborhood's latitude and longitude values
neighborhood_latitude = downtown_toronto_data.loc[0, 'Latitude']
neighborhood_longitude = downtown_toronto_data.loc[0, 'Longitude']
neighborhood_name = downtown_toronto_data.loc[0, 'Neighbourhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Creating function for making get request to retrieve venues to the given location.

In [25]:
#Making GET request to retreive venues to the given location
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
#Showing venues around Downtown Toronto, what are the neighborhoods
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighbourhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [27]:
#Checking the size of Downtown Toronto dataframe
print('Shape of Downtown Toronto venues is:', downtown_toronto_venues.shape)
print()
downtown_toronto_venues.head()

Shape of Downtown Toronto venues is: (1222, 7)



,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [28]:
#Grouping venues by 'Neighborhood' column
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [29]:
#Showing the number of different categories in venues
print('There are {} unique categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 208 unique categories.


### Analyzing the Neighborhood

In this part, I'll try to find how often palces are shown and amog them I'll try to get the most visited places, so to be able to analyze which places are best for visiting. The most visiting places have tendancy to offer good time there, so people like to go there and to recomend to friends.
ON start it is definied venu category and neighborhood, and then are definied most common venues. After that, data are sorted and grouped to show neighbohood with places with most common venues.

In [30]:
#Calculating mean of the frequency of occurrence of each category and picked top  venues on that basis for each neighborhood. 
#That means the top venues are showing the more visited places

#one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#Adding neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

#Moving neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#Showing shape
print('Shape of one hot encoding of Downtown Toronto:', downtown_toronto_onehot.shape)

Shape of one hot encoding of Downtown Toronto: (1222, 208)


In [32]:
#Reseting index
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.017857,0.000000,0.000000,0.00,0.00,0.017857,0.00000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000
2,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.031746,0.000000,0.000000,0.00,0.00,0.015873,0.00000,0.000000,0.015873
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000
4,Church and Wellesley,0.025316,0.012658,0.000000,0.000000,0.000000,0.0,0.000000,0.012658,0.000000,...,0.000000,0.012658,0.012658,0.012658,0.00,0.00,0.000000,0.00000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.010000,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.000000,0.010000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.010000,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.010000,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.01000,0.010000,0.010000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.015625,0.000000,0.000000,0.000000,0.00,0.00,0.062500,0.00000,0.046875,0.015625


In [33]:
print('Shape of Downtown Toronto grouped is:', downtown_toronto_grouped.shape)

Shape of Downtown Toronto grouped is: (19, 208)


In [34]:
#Showing the top 5 most common venues

num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3  Seafood Restaurant  0.04
4            Pharmacy  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4       Coffee Shop  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2      Sandwich Place  0.06
3  Italian Restaurant  0.05
4     Thai Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3           Restaurant  0.04
4        

In [35]:
#Puting venues into a pandas dataframe
#Defining of function to sort the venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
#Creating the new dataframe and displaying the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

#Creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Pharmacy,Gourmet Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Rental Car Location,Coffee Shop,Harbor / Marina,Airport Food Court,Airport
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Middle Eastern Restaurant,Indian Restaurant
3,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Coffee Shop,Restaurant,Italian Restaurant,Athletics & Sports,Candy Store
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant


### Cluster neighborhoods

In this part, using k-means, I'll  find cluster in the neighborhood, and try to put them into 5 clusters and also try to give all of them their category. Given data, I'll fit within groupd cluster. Map should show ali given point of interest that have venue in high number.

In [37]:
#Runing k-means to cluster the neighborhood into 5 clusters

#Setting number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

#Runing k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

#Checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 4, 3, 2, 2, 2, 2, 2, 2], dtype=int32)

In [38]:
#Creating a new dataframe that includes the cluster as well as the top venues for each neighborhood.
#Adding clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
downtown_toronto_merged = downtown_toronto_data

#Merging downtown_toronto_grouped with downtown_toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
downtown_toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Theater,Breakfast Spot,Pub,Café,Performing Arts Venue,Dessert Shop,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Bar,Bank,Burger Joint,Café,Italian Restaurant,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Clothing Store,Gym,Hotel,Department Store,Creperie
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Pharmacy,Gourmet Shop


In [39]:
#Visualizing the resulting clusters

#Creating map of Downtown Toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

#Seting color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighbourhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Checking each cluster for different categories

In [40]:
#Cluster 1 - the category of airport area, harbor/marina area, sculpture garden
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Rental Car Location,Coffee Shop,Harbor / Marina,Airport Food Court,Airport


In [41]:
#Cluster 2 - the category of shops and similar area
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Trail,Playground,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [42]:
#Cluster 3 - the category of stores, restorant and entertaiment
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Bakery,Park,Theater,Breakfast Spot,Pub,Café,Performing Arts Venue,Dessert Shop,Distribution Center
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel
3,Downtown Toronto,2,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Clothing Store,Gym,Hotel,Department Store,Creperie
4,Downtown Toronto,2,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Pharmacy,Gourmet Shop
7,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Deli / Bodega,Thai Restaurant,Bakery,Clothing Store,Gym,Cosmetics Shop,Steakhouse
8,Downtown Toronto,2,Coffee Shop,Aquarium,Hotel,Café,Sporting Goods Shop,Italian Restaurant,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint
9,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Bakery,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant
10,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega
11,Downtown Toronto,2,Café,Japanese Restaurant,Bar,Bookstore,Bakery,Restaurant,Italian Restaurant,Beer Bar,Beer Store,Sandwich Place
12,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Mexican Restaurant,Vietnamese Restaurant,Grocery Store,Farmers Market,Gaming Cafe,Bakery


In [43]:
#Cluster 4 - the category of park area and shops
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Grocery Store,Café,Park,Baby Store,Nightclub,Coffee Shop,Restaurant,Italian Restaurant,Athletics & Sports,Candy Store


In [44]:
#Cluster 5 - the category of cafes, bars, pub, bakery, restaurants, stores
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Bar,Bank,Burger Joint,Café,Italian Restaurant,Japanese Restaurant
5,Downtown Toronto,4,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Middle Eastern Restaurant,Indian Restaurant
